In [9]:
import numpy as np
import theano
import theano.tensor as T
import csv as csv 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

#Открываем и читаем csv файл
train_df = pd.read_csv('./data/train.csv')
print train_df.shape
train_df.Prediction.value_counts(normalize=True)
train_df.head()

(110163, 3)


,ID,Reviews_Summary,Prediction
0,239071,Michigan Cherries,1
1,466160,Great Product,1
2,397133,Ovaltine,1
3,297146,~,1
4,292685,Love it!,1


In [10]:
review_summaries = list(train_df['Reviews_Summary'].values)
review_summaries = [l.lower() for l in review_summaries]
vectorizer = TfidfVectorizer()
tfidfed = vectorizer.fit_transform(review_summaries)
from sklearn.cross_validation import train_test_split
X = tfidfed#<используйте векторные представления текстов, полученные на предыдущем шаге>
y = train_df.Prediction.values
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

In [53]:
a = np.ndarray([5,5])

[[  6.93660740e-310   4.07583051e-316   6.93660761e-310   6.93655357e-310
    6.93660757e-310]
 [  6.93655357e-310   6.93660761e-310   6.93655357e-310   6.93660757e-310
    6.93655357e-310]
 [  6.93660761e-310   6.93655357e-310   6.93660757e-310   6.93655357e-310
    6.93660761e-310]
 [  6.93655357e-310   6.93660757e-310   6.93655357e-310   6.93660761e-310
    6.93655357e-310]
 [  6.93660757e-310   6.93655357e-310   6.93660761e-310   6.93655357e-310
    1.02765654e-321]]
[[  6.93660740e-310   4.07583051e-316   6.93660761e-310   6.93655357e-310
    6.93660757e-310]]


In [12]:
print X_train.shape, X_test.shape, y_test.shape, y_train.shape

(77114, 16230) (33049, 16230) (33049,) (77114,)


In [14]:
X_train_sample = X_train[:10000]
y_train_sample = y_train[:10000]     

In [35]:
# Декларируем символьные переменные Theano 
x = T.matrix("x", dtype = 'float64')
y = T.vector("y", dtype = 'int64')
w = theano.shared(np.random.randn(X_train.shape[1]) * 2, name="w")
b = theano.shared(0., name="b")

In [40]:
def lol(x,w,b):
    return 1 / (1 + T.exp(-T.dot(x, w) - b))

In [41]:
# Создаем «выражение» Theano
p_1 = lol(x,w,b)           # Вероятность того, что результат равен  1
prediction = p_1 > 0.5                                    # Порог для прогнозирования
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Функция ошибки для перекрестной энтропии
cost = xent.mean() + 0.01 * (w ** 2).sum()  # Стоимость минимизации
gw,gb = T.grad(cost, [w, b])                          # Рассчитываем градиент стоимости

In [42]:
# Компилируем «выражение» Theano
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)


In [49]:
# Тренировка модели
batch_size = 200
for i in range(10000):
    idx = np.random.choice(X.shape[0], batch_size)
    X_batch = X_train_sample[i:i + batch_size]
    y_batch = y_train_sample[i:i + batch_size]
    pred, err = train(X_batch.todense(), y_batch)
    if(not i % 1000):
        print np.linalg.norm(err)

9.26849984583
9.16810697949
9.14702969877
9.13859716213
9.22047007189
9.11165239864
9.22642044187
9.12993195946
9.11480689854
9.15361845919


KeyboardInterrupt: 

In [54]:
%time print lol

<function lol at 0x7fb0d7b54b90>
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 108 µs
